In [8]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC

In [11]:
train_data = np.load('自动驾驶疲劳检测挑战赛公开数据-更新/train.npy')
test_data = np.load('自动驾驶疲劳检测挑战赛公开数据-更新/test.npy')
train_label = pd.read_csv('自动驾驶疲劳检测挑战赛公开数据-更新/train_label.csv', header=None)

In [15]:
train_data.shape, test_data.shape

((10000, 64, 64, 3), (15000, 64, 64, 3))

In [ ]:
m = LinearSVC()
m.fit(train_data.reshape(10000, -1), train_label)

In [17]:
pd.DataFrame(m.predict(test_data.reshape(15000, -1))).to_csv('submit.csv', index=None, header=None)